# The important parts of the code

## Command imports

In [1]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
import glob
import random
import math
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


## Import mse IDs 

In [2]:
#subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
subjects = ["mse1406"]
all_sub_results = [["mseID",
                   "lesion number and type",      
                   "number of subcortical lesions",
                   "number of juxtacortical lesions",
                   "number of periventricular lesions",
                   "number of infratentorial lesions"]]
print subjects[:5]

['mse1406']


## Set variables to lesion/segmentation files, freeview cmdline copy and paste

In [3]:
for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[0]):
    print les_file
for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[0]):
    print seg_file
for gm_file in glob.glob("/data/henry6/PBR/surfaces/*%s*/mri/ribbon.mgz" % subjects[0]):
    print gm_file

print
print "Freeview link:"
print 
print "freeview", seg_file, les_file, gm_file

/data/henry7/PBR/subjects/mse1406/lesions_manual/ms172-mse1406-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1406/masks/ms172-mse1406-002-AX_T1_3D_IRSPGR/segmentation.nii.gz
/data/henry6/PBR/surfaces/ms172-mse1406-002-AX_T1_3D_IRSPGR/mri/ribbon.mgz

Freeview link:

freeview /data/henry7/PBR/subjects/mse1406/masks/ms172-mse1406-002-AX_T1_3D_IRSPGR/segmentation.nii.gz /data/henry7/PBR/subjects/mse1406/lesions_manual/ms172-mse1406-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz /data/henry6/PBR/surfaces/ms172-mse1406-002-AX_T1_3D_IRSPGR/mri/ribbon.mgz


## Obtain affines 

In [4]:
# les corresponds to lesion file
# 
# seg corresponds to segmentation file (important for midbrain and ventricular structures, 
# plus real space conversion)
#
# gm corresponds to ribbon file that gives gray matter coordinates

les_img = nib.load(les_file)
les_img.dataobj
seg_img = nib.load(seg_file)
seg_img.dataobj
gm_img = nib.load(gm_file)
gm_img.dataobj

les_data = les_img.get_data()
print les_data.shape
seg_data = seg_img.get_data()
print seg_data.shape
gm_data = gm_img.get_data()
print gm_data.shape

les_affine = les_img.get_affine()
print les_affine
gm_affine = gm_img.get_affine()
print gm_affine
seg_affine = seg_img.get_affine()
print seg_affine
inv_seg_affine = np.linalg.inv(seg_affine)
print inv_seg_affine

(256, 256, 149)
(256, 256, 256)
(256, 256, 256)
[[  -0.9375        0.            0.          121.8730011 ]
 [   0.            0.9375        0.          -86.33650208]
 [   0.            0.            1.          -60.81729889]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.8730011 ]
 [   0.            0.            1.          -94.33650208]
 [   0.           -1.            0.          157.18270111]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          129.8730011 ]
 [   0.            0.            1.          -94.33650208]
 [   0.           -1.            0.          157.18270874]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          129.8730011 ]
 [  -0.           -0.           -1.          157.18270874]
 [   0.            1.            0.           94.33650208]
 [   0.            0.            0.            1.        ]]


## Set labels for structure's coordinates

In [5]:
#lesion labels
les_labels, n_les_labels = label(les_data==[1])

#segmentation - brainstem labels
seg_brainstem_labels, n_seg_brainstem_labels = label(seg_data==[16])
seg_lcerebellumcortex_labels, n_seg_lcerebellumcortex_labels = label(seg_data==[8])
seg_rcerebellumcortex_labels, n_seg_rcerebellumcortex_labels = label(seg_data==[47])
seg_lcerebellumwm_labels, n_seg_lcerebellumwm_labels = label(seg_data==[7])
seg_rcerebellumwm_labels, n_seg_rcerebellumwm_labels = label(seg_data==[46])

#segmentation - lateral ventricle labels
seg_llv_labels, n_seg_llv_labels = label(seg_data==[4])
seg_rlv_labels, n_seg_rlv_labels = label(seg_data==[43])

#gm labels
gm_lh_labels, n_gm_lh_labels = label(gm_data==3)
gm_rh_labels, n_gm_rh_labels = label(gm_data==42)

### debugging tools for labels

print n_les_labels #prints number of lesion labels

print n_seg_brainstem_labels #prints number of labels in brainstem

print n_gm_lh_labels #prints number of labels in gray matter

print n_seg_llv_labels #prints number of labels in ventricle

print np.bincount(les_labels.ravel()) #prints volumes of lesions

print np.bincount(seg_lcerebellumwm_labels.ravel()) #prints volume of left cerebellum white matter

print np.bincount(gm_lh_labels.ravel()) #prints volume of left hemisphere gray matter

print np.bincount(seg_llv_labels.ravel()) #prints volume of left hemisphere ventricle

## Function that gets coordinates from labels

In [6]:
def get_label_coord(labels,num_labels):
    all_label_coords = []
    if num_labels >= 1:
        for count in range(1, num_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
    else:
        x,y,z = np.nonzero(labels==1)
        for count in range(len(x)):
            all_label_coords.append([x[count],y[count],z[count]])
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_label_coords

## Function that converts coordinates to real space

In [7]:
def get_rs_coord(coordinates,affine):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(affine, [coordinates[count][count2][0],
                                           coordinates[count][count2][1], 
                                           coordinates[count][count2][2],
                                           1])
            rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

## Function that converts coordinates to segmentation file coordinates

In [8]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                coordinates[count][count2][1], 
                                                coordinates[count][count2][2],
                                                1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            #seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

## Generate midbrain coordinates into a variable

In [9]:
bs = get_label_coord(seg_brainstem_labels,n_seg_brainstem_labels)
lcc = get_label_coord(seg_lcerebellumcortex_labels,n_seg_lcerebellumcortex_labels)
lcw = get_label_coord(seg_lcerebellumwm_labels,n_seg_lcerebellumwm_labels)
rcc = get_label_coord(seg_rcerebellumcortex_labels,n_seg_rcerebellumcortex_labels)
rcw = get_label_coord(seg_rcerebellumwm_labels,n_seg_rcerebellumwm_labels)

midbrain_tmp = []
midbrain = []

if len(bs) != 1:
    midbrain_tmp.append(bs[1])
else:
    midbrain_tmp.append(bs[0])

if len(lcc) != 1:
    midbrain_tmp.append(lcc[1])
else:
    midbrain_tmp.append(lcc[0])

if len(lcw) != 1:
    midbrain_tmp.append(lcw[1])
else:
    midbrain_tmp.append(lcw[0])

if len(rcc) != 1:
    midbrain_tmp.append(rcc[1])
else:
    midbrain_tmp.append(rcc[0])

if len(rcw) != 1:
    midbrain_tmp.append(rcw[1])
else:
    midbrain_tmp.append(rcw[0])

for x in range(len(midbrain_tmp)):
    for y in range(len(midbrain_tmp[x])):
        midbrain.append(midbrain_tmp[x][y])
        

## Generate gray matter coordinates into a variable

In [10]:
gm = []
lhcoord = get_label_coord(gm_lh_labels,n_gm_lh_labels)
rhcoord = get_label_coord(gm_rh_labels,n_gm_rh_labels)
lhcoord_seg = get_seg_coord(get_rs_coord(lhcoord,gm_affine))
rhcoord_seg = get_seg_coord(get_rs_coord(rhcoord,gm_affine))

for x in range(len(lhcoord_seg[0])):
    gm.append(lhcoord_seg[0][x])
for x in range(len(rhcoord_seg[0])):
    gm.append(rhcoord_seg[0][x])

## Generate ventricular coordinates into a variable

In [11]:
ventricles_tmp = []
ventricles = []
vlhcoord = get_label_coord(seg_llv_labels,n_seg_llv_labels)
vrhcoord = get_label_coord(seg_rlv_labels,n_seg_rlv_labels)

if len(vlhcoord) != 1:
    ventricles_tmp.append(vlhcoord[1])
else:
    ventricles_tmp.append(vlhcoord[0])
if len(vrhcoord) != 1:
    ventricles_tmp.append(vrhcoord[1])
else:
    ventricles_tmp.append(vrhcoord[0])

for x in range(len(ventricles_tmp)):
    for y in range(len(ventricles_tmp[x])):
        ventricles.append(ventricles_tmp[x][y])

## Generate lesion coordinates into a variable

In [12]:
lesions_les = get_label_coord(les_labels,n_les_labels)
lesions = get_seg_coord(get_rs_coord(lesions_les,les_affine))
lesions_seg = lesions

## Generate random coordinates from each structure

In [14]:
gm_test = []
mb_test = []
v_test = []
randcount = 5000 #variable count of 

def random_samp(lst,count):
    randlst = []
    for x in range(count):
        randlst.append(lst[random.randrange(0,len(lst))])
    return randlst

gm_test = random_samp(gm,randcount)
mb_test = random_samp(midbrain,randcount)
v_test = random_samp(ventricles,randcount)

## Function that takes average of lesion coordinates

In [15]:
def average_func(coordinates):
    sumx=0;sumy=0;sumz=0
    for count in range(len(coordinates)):
        sumx += coordinates[count][0]
        sumy += coordinates[count][1]
        sumz += coordinates[count][2]
    average_x = sumx / len(coordinates)
    average_y = sumy / len(coordinates)
    average_z = sumz / len(coordinates)
    #print count, averages
    return [average_x, average_y, average_z]

les_averages = []
for x in range(len(lesions)):
    les_averages.append(average_func(lesions[x]))
print les_averages

print les_averages[1]
print len(gm)
print len(midbrain)
print len(ventricles)

[[88, 124, 96], [99, 134, 79], [100, 133, 147], [101, 143, 62], [102, 151, 64], [108, 141, 158], [110, 156, 148], [115, 105, 141], [126, 193, 104], [136, 121, 108], [145, 141, 156], [144, 96, 132], [146, 131, 68], [150, 123, 90], [152, 138, 60], [155, 150, 153], [159, 154, 69]]
[86, 158, 53]
[113, 181, 80]
[99, 134, 79]
442946
130947
45254


## Euclidean distance determination function 

In [16]:
def dist_det(struc_coord, lesion_coord):
    dist_list = []
    for x in range(len(struc_coord)):
        dist_list.append(math.sqrt((struc_coord[x][0] - lesion_coord[0]) ** 2 +
                                   (struc_coord[x][1] - lesion_coord[1]) ** 2 +
                                   (struc_coord[x][2] - lesion_coord[2]) ** 2))
    return dist_list

## Classifying lesions by lowest Euclidean distance

In [17]:
min_val_jux = 3.0    #threshold for juxtacortical lesion's distance to gray matter
min_val_per = 6.0    #threahold for periventricular lesion's distance to ventricle
min_val_mb = 6.0     #threshold for infratentorial lesion's distance to midbrain
les_type = []

#if we wish to use random numbers:

#for x in range(len(les_averages)):
#    gm_min = np.min(dist_det(gm_test,les_averages[x]))
#    mb_min = np.min(dist_det(mb_test,les_averages[x]))
#    v_min = np.min(dist_det(v_test,les_averages[x]))
#    if gm_min <= min_val_jux and gm_min <= mb_min and gm_min <= v_min:
#        lesion_type = "juxtacortical"
#    elif v_min <= min_val_per and v_min <= mb_min:
#        lesion_type = "periventricular"
#    elif mb_min <= min_val_mb:
#        lesion_type = "infratentorial"
#    else:
#        lesion_type = "subcortical"
#    print les_averages[x], x, lesion_type, gm_min, mb_min, v_min
#
#print    

#if we wish to use all plots in each structure
for x in range(len(les_averages)):
    gm_min = np.min(dist_det(gm,les_averages[x]))
    mb_min = np.min(dist_det(midbrain,les_averages[x]))
    v_min = np.min(dist_det(ventricles,les_averages[x]))
    if gm_min <= min_val_jux and gm_min <= mb_min and gm_min <= v_min:
        lesion_type = "juxtacortical"
    elif v_min <= min_val_per and v_min <= mb_min:
        lesion_type = "periventricular"
    elif mb_min <= min_val_mb:
        lesion_type = "infratentorial"
    else:
        lesion_type = "subcortical"
    print les_averages[x], x, lesion_type, gm_min, mb_min, v_min
    les_type.append(lesion_type)
    

[88, 124, 96] 0 juxtacortical 0.0 43.4741302386 17.2626765016
[99, 134, 79] 1 periventricular 6.40312423743 32.3882694814 4.12310562562
[100, 133, 147] 2 juxtacortical 3.0 51.0392006207 11.0453610172
[101, 143, 62] 3 juxtacortical 2.44948974278 27.3861278753 13.0384048104
[102, 151, 64] 4 juxtacortical 0.0 19.3390796058 12.2474487139
[108, 141, 158] 5 periventricular 5.83095189485 52.7067358124 3.31662479036
[110, 156, 148] 6 juxtacortical 1.0 36.7831483155 6.40312423743
[115, 105, 141] 7 juxtacortical 2.82842712475 60.2992537267 21.8403296678
[126, 193, 104] 8 infratentorial 21.3775583264 0.0 44.2944691807
[136, 121, 108] 9 periventricular 4.12310562562 33.2415402772 2.0
[145, 141, 156] 10 periventricular 6.16441400297 50.219518118 0.0
[144, 96, 132] 11 juxtacortical 0.0 64.8999229584 27.4590604355
[146, 131, 68] 12 periventricular 3.46410161514 30.0166620396 5.38516480713
[150, 123, 90] 13 periventricular 7.81024967591 35.5949434611 2.2360679775
[152, 138, 60] 14 subcortical 4.123105

## Append results to running list 

In [18]:
results = []
sub_count, inf_count, jux_count, per_count, err_count = 0, 0, 0, 0, 0
for count in range(len(les_type)):
    results.append([count+1, les_type[count]])
    if les_type[count] == "subcortical":
        sub_count += 1
    elif les_type[count] == "infratentorial":
        inf_count += 1
    elif les_type[count] == "juxtacortical":
        jux_count += 1
    elif les_type[count] == "periventricular":
        per_count += 1
all_sub_results.append([subjects[0], 
                        results, 
                        sub_count, 
                        jux_count,
                        per_count,
                        inf_count]
                      )
print all_sub_results

[['mseID', 'lesion number and type', 'number of subcortical lesions', 'number of juxtacortical lesions', 'number of periventricular lesions', 'number of infratentorial lesions'], ['mse1406', [[1, 'juxtacortical'], [2, 'periventricular'], [3, 'juxtacortical'], [4, 'juxtacortical'], [5, 'juxtacortical'], [6, 'periventricular'], [7, 'juxtacortical'], [8, 'juxtacortical'], [9, 'infratentorial'], [10, 'periventricular'], [11, 'periventricular'], [12, 'juxtacortical'], [13, 'periventricular'], [14, 'periventricular'], [15, 'subcortical'], [16, 'juxtacortical'], [17, 'periventricular']], 1, 8, 7, 1]]
